In [1]:
import pandas as pd
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
import h5py
import os
import sys
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from statistics import mean , stdev
import pickle
from sklearn.metrics import confusion_matrix
import xgboost as xgb
import random
from rdkit import RDLogger
import argparse
import warnings
warnings.filterwarnings("ignore")
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

In [2]:
def feature_template_df():
    '''Creates a template dataframe 
       containing all the feature columns''' 
    global no_of_rxns_thres
    feature_data_columns = []
    for n in range(no_of_rxns_thres):
        smarts = "Rxn"+str(n+1)+"_SMARTS"
        rxn_delta_g = "Rxn"+str(n+1)+"_DeltaG"
        rxn_rule_score = "Rxn"+str(n+1)+"_Rule_Score"
        feature_data_columns.extend([smarts, rxn_delta_g, rxn_rule_score])
    feature_data_columns.extend(["Pathway_Delta_G", "Pathway_Flux", "Pathway_Score", "Round1"])
    #print(feature_data_columns)
    feature_data = pd.DataFrame(  columns = feature_data_columns , index = None)
    return feature_data

def loop(i , temp, data):
    '''Returns the indices of all the reactions
       for a pathway in the dataset''' 
    temp_list = []
    break_index = None 
    flag = True
    j = 1
    for index in range(i, len(data)):
        if temp == data.loc[index,'Pathway Name'] and data.loc[index,'Reaction'] == "RP"+str(j):
            j =j+1
            temp_list.append(index)
            if index+1 == len(data):
                flag = False
        else:
            break_index = index
            break
    return temp_list , break_index , flag


def pathways_index_list(data):
    '''Returns the indices of all the reactions
       for each of the pathways in dataset''' 
    pathways_index_list = []
    i = 0 
    flag = True
    while flag:
        temp = data.loc[i,'Pathway Name']
        temp_list, i , flag = loop( i , temp, data)
        pathways_index_list.append(temp_list)

    return pathways_index_list


def transform_into_pathway_features(data, scores, flag):
    '''Generates the dataframe containing
       all the features. 
       data and scores ate the 2 inputs files
       The reactions are represented in SMILES'''

    global no_of_rxns_thres
    df = feature_template_df()
    pathways_list = pathways_index_list(data)
    #print(pathways_list)
    #print(len(pathways_list))
    drop_list = []
    print("Transforming into pathway features...")
    for count_p, rxn_list in tqdm(enumerate(pathways_list)) :
        if len(rxn_list) > 10:
            drop_list.append(count_p)
            continue
        # At the level of each reaction reading data file
        for n , index in enumerate(rxn_list):
            #print(index)
            smarts = "Rxn"+str(n+1)+"_SMARTS"
            rxn_delta_g = "Rxn"+str(n+1)+"_DeltaG"
            rxn_rule_score = "Rxn"+str(n+1)+"_Rule_Score"
            df.loc[count_p, smarts] = data.loc[index,'Reaction Rule']
            df.loc[count_p, rxn_delta_g] = data.loc[index,'Normalised dfG_prime_m']
            df.loc[count_p, rxn_rule_score] = data.loc[index,'Rule Score']
        # At the level of pathway reading scores file
        df.loc[count_p, "Pathway_Delta_G"] = scores.loc [count_p, 'dfG_prime_m'] 
        df.loc[count_p, "Pathway_Flux"] = float( scores.loc[count_p,'FBA Flux'].split(';')[1] ) 
        df.loc[count_p, "Pathway_Score"] = scores.loc[count_p,'Global Score']
        df.loc[count_p, "Lit"] = scores.loc[count_p,'Lit']
        df.loc[count_p, "Round1"] = scores.loc[count_p,'Round1']
    df = df.drop(drop_list)
    df = df.fillna(0)
    if flag:
        df = df[~(df.Round1 < 0)]
        df["Round1"][df['Round1'] > 0] = 1
        #df.to_csv("raw_df.csv")
        df["Round1_OR"] = df["Round1"]
        df = shuffle(df, random_state = 42).reset_index(drop =True)
        for row in range(len(df)):
            if  df.loc[row , "Lit"] == 1 :
                df.loc[row , "Round1_OR"] = 1
        #df.to_csv("raw_df_csv", index = None)  
        #print(df)
    else :
        df["Round1_OR"] = df["Round1"]
    return df

def features_encoding (df, flag):
    '''Creates a HDF5 file containing
       all the features
       Rnx features are encoded in fingerprints'''
    no_of_rxns = 10
    fp_len = 4096
    rxn_len = fp_len + 2
    pathway_len = 3
    y_len = 1

    if flag == "train":
        sys.exit('Encoding feature for training data not available file data_train.h5 must be present in models folder')
    elif flag == "predict":
        path = 'models/data_predict.h5'
        print("Encodining features for the Test set......")
    if os.path.exists(path):
        os.remove(path)
    f=h5py.File(path, "w")
    dset = f.create_dataset('data', (  0, (rxn_len*no_of_rxns + pathway_len + y_len)),dtype='i2',maxshape=(None,(rxn_len*no_of_rxns + pathway_len + y_len)), compression='gzip')

    for row in tqdm(range(len(df))):
        pathway_rxns = np.array([]).reshape(0, rxn_len * no_of_rxns)
        rxns_list = []
        for rxn_no_ in range(no_of_rxns):
            
            rxn_smiles_index = rxn_no_ * 3
            rxn_dg_index = (rxn_no_ + 1)* 3 -2
            rxn_rule_score_index = (rxn_no_ + 1)* 3 - 1
        
            if  str(df.iloc[row , rxn_smiles_index]) != '0':
                #print(df.iloc[row , rxn_smiles_index])
                rxn_smiles = df.iloc[row , rxn_smiles_index]
                rxn_smiles_list = rxn_smiles.split(">>")
                #print(len(rxn_smiles_list))

                if len(rxn_smiles_list) == 2:

                    sub_smiles = rxn_smiles_list[0]
                    sub_m= Chem.MolFromSmiles(sub_smiles)
                    #print(m)
                    sub_fp = AllChem.GetMorganFingerprintAsBitVect(sub_m, 2, nBits = 2048)
                    sub_arr = np.array([])
                    DataStructs.ConvertToNumpyArray(sub_fp, sub_arr)
                    sub_fp= sub_arr.reshape(1,-1)

                    pro_smiles = rxn_smiles_list[1]
                    pro_m= Chem.MolFromSmiles(pro_smiles)
                    #print(m)
                    pro_fp = AllChem.GetMorganFingerprintAsBitVect(pro_m, 2, nBits = 2048)
                    pro_arr = np.zeros((1,))
                    DataStructs.ConvertToNumpyArray(pro_fp, pro_arr)
                    pro_fp= pro_arr.reshape(1,-1)
                    rxn_fp = np.concatenate([sub_fp , pro_fp]).reshape(1, -1)

                elif len(rxn_smiles_list) < 2:
                     
                    pro_smiles = rxn_smiles_list[0]
                    #print(pro_smiles)
                    pro_m= Chem.MolFromSmiles(pro_smiles)
                    #print(pro_m)
                    pro_fp = AllChem.GetMorganFingerprintAsBitVect(pro_m, 2, nBits = fp_len) # JLF: not good !!
                    pro_arr = np.zeros((1,))
                    DataStructs.ConvertToNumpyArray(pro_fp, pro_arr)
                    rxn_fp= pro_arr.reshape(1,-1)
                else:
                    print("There is a problem with the number of components in the reaction")

            else:
                rxn_fp = np.zeros(fp_len).reshape(1,-1)

            rxn_dg = df.iloc[row , rxn_dg_index].reshape(1,-1)
            rxn_rule_score = df.iloc[row , rxn_rule_score_index].reshape(1,-1)
            rxns_list.extend([rxn_fp, rxn_dg, rxn_rule_score])
            #print(rxn_rule_score)

        pathway_rxns = np.concatenate(rxns_list , axis = 1).reshape(1,-1)
        pathway_dg = df.loc[row, "Pathway_Delta_G"].reshape(1,-1)
        pathway_flux = df.loc[row, "Pathway_Flux"].reshape(1,-1)
        pathway_score = df.loc[row, "Pathway_Score"].reshape(1,-1)
        pathway_y = df.loc[row, "Round1_OR"].reshape(1,-1)
        feature = np.concatenate((pathway_rxns, pathway_dg, pathway_flux, pathway_score, pathway_y), axis =1)
        dset.resize(dset.shape[0]+feature.shape[0], axis=0) 
        dset[-feature.shape[0]:]= feature
        #print(pathway_flux)

    return dset


def transform_to_matrix(dset):
    ''''Transforms the prediction dataset into
        an appropriate matrix which is suitable for
        XGBoost'''
    X_test = dset[:,:-1]
    Y_test = dset[:, -1]
    
    if not os.path.exists('models/model.pickle'):
        sys.exit('model.pickle not found')
    else:
        trained_model = pickle.load(open('models/model.pickle' ,'rb'))

    dset_matrix = xgb.DMatrix(X_test, label = Y_test)
    trained_model_score =  trained_model.predict(dset_matrix)
    trained_model_score_1 = trained_model_score[:, 1].reshape(-1,1)
    X_test = np.concatenate((X_test, trained_model_score_1), axis = 1)
    dset_matrix = xgb.DMatrix(X_test)

    return  dset_matrix
    

###############################################################
def score_prediction(features_dset_train, features_dset_pred):

    stdev_ = []
    mean_ = []
    pb1_mean = []
    pb1_stdev = []
    print(features_dset_pred)
    n_predictions = np.array([[]])
    print("Predicting n times...")
    for model_number,  n in tqdm( enumerate([ 0, 10, 20, 30, 40, 50, 60, 70,80, 90])): ##########################
        modelfile = "models/model" + str(model_number )+ ".pickle"
        if not os.path.exists(modelfile):
            sys.exit('modelfile not found')
        model = pickle.load( open(modelfile, 'rb')) ############
        df_test_matrix = transform_to_matrix(features_dset_pred) 
        prediction = model.predict(df_test_matrix)
        pb_1 = prediction[:, 1].reshape(-1, 1)
        prediction = np.asarray([np.argmax(line) for line in prediction]).reshape(-1, 1)
        if n_predictions.shape[1] == 0 :
            n_predictions = prediction
            n_pb_1 = pb_1
        else:
            n_predictions = np.concatenate((n_predictions , prediction), axis = 1)
            n_pb_1 = np.concatenate((n_pb_1, pb_1), axis = 1)

    for row in range(len(n_predictions)):
        line = n_predictions[row, :].tolist()
        line_pb1 = n_pb_1[row, :].tolist()

        mean_.append(mean(line))
        stdev_.append(stdev(line))
        pb1_mean.append(mean(line_pb1))
        pb1_stdev.append(stdev(line_pb1))

    mean_stdev = pd.DataFrame( { 'mean' : mean_ , 'stdev' : stdev_ , 'Prob1_mean': pb1_mean , 'Prob1_stdev' : pb1_stdev})
    mean_stdev.to_csv("mean_stdev_.csv")


In [3]:
##############################################################
no_of_rxns_thres = 10

"""
parser = argparse.ArgumentParser()
parser.add_argument('-trdf', '--train_data_file', required=True, type=str)
parser.add_argument('-trsf', '--train_score_file', required=True, type=str)
parser.add_argument('-ttdf', '--test_data_file', required=True, type=str)
parser.add_argument('-ttsf', '--test_score_file', required=True, type=str)
args = parser.parse_args()
train_data_file =  args.train_data_file
train_score_file = args.train_score_file
test_data_file = args.test_data_file
test_score_file = args.test_score_file
"""

# Load training and test data
if not os.path.exists('models/data_train.h5'):
    sys.exit('models/data_train.h5 not found')
f=h5py.File('models/data_train.h5', "r")
features_dset_train = f["data"]

data_test   = 'reactions_all.csv'
scores_test = 'pathways_all.csv'
if not os.path.exists(data_test):
    sys.exit('reactions_features not found')
else:
    data_test = pd.read_csv(data_test)
if not os.path.exists(scores_test):
    sys.exit('pathways_features not found')
else:
    scores_test = pd.read_csv(scores_test)   
print("Number of pathways : ",len(scores_test))
print("Total number of reactions  : ", len(data_test))

# Encoding and prediction
df_test = transform_into_pathway_features(data_test, scores_test, False)
features_dset_pred  = features_encoding(df_test, "predict")
score_prediction(features_dset_train, features_dset_pred)
print("Mean - Stdev stats is saved")

f.close()
################################################################

1it [00:00, 181.05it/s]
100%|██████████| 1/1 [00:00<00:00, 41.64it/s]
0it [00:00, ?it/s]

Number of pathways :  1
Total number of reactions  :  7
Transforming into pathway features...
Encodining features for the Test set......
<HDF5 dataset "data": shape (1, 40984), type "<i2">
Predicting n times...


10it [00:02,  4.57it/s]

Mean - Stdev stats is saved
